In [1]:
import pandas as pd

### 1. update all data

In [ ]:
data_url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
raw_data = pd.read_csv(data_url, sep=",")

In [ ]:
raw_data.head()

In [ ]:

raw_data.columns

In [ ]:
processed_data = raw_data.loc[:, ["date", "location", "total_cases", "people_vaccinated"]]

In [ ]:
processed_data.head()

In [ ]:
processed_data.dtypes

### 2. Process pipeline


In [ ]:
import numpy as np

from datetime import datetime

In [ ]:
def store_relational_JH_data():
    ''' Transformes the COVID data in a relational data set

    '''
    
    data_url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
    pd_raw = pd.read_csv(data_url, sep=",")
    
    pd_relational_model = pd_raw.loc[:, ["date", "location", "total_cases", "people_vaccinated"]]
    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')
    pd_relational_model=pd_relational_model.rename(columns={'total_cases':'confirmed',
                      'location':'country'})

    pd_relational_model.to_csv('/Users/pawnesh/ws/covid_data_science/covid_19_data_stats/data/raw/COVID-19/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    print(' Latest date is: '+str(max(pd_relational_model.date)))
    print(pd_relational_model.head())
    print(pd_relational_model.dtypes)

In [ ]:
store_relational_JH_data()

In [ ]:
#!pip install scikit-learn

### 3 Filter and Doubling Rate Calculation

In [3]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    try:
        result=signal.savgol_filter(np.array(filter_in),
                               window, # window size used for filtering
                               1)
    except (UnboundLocalError, ValueError) as e:
        print(e)
        result = filter_in
    
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten
    
    pd_filtered_result=df_output[['country',filter_on]].groupby(['country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())
    
    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'
    

    pd_DR_result= df_input.groupby(['country']).apply(rolling_reg,filter_on).reset_index()
    #import pdb; pdb.set_trace()
    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_1':'index'})
    #import pdb; pdb.set_trace()
    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output

In [4]:
test_data_reg=np.array([2,4,6])
result=get_doubling_time_via_regression(test_data_reg)
print('the test slope is: '+str(result))

pd_JH_data=pd.read_csv('/Users/pawnesh/ws/covid_data_science/covid_19_data_stats/data/raw/COVID-19/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data = pd_JH_data.reset_index()
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

#test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
#                  (pd_JH_data['country']=='Germany'))]

pd_result_larg=calc_filtered_data(pd_JH_data)
pd_result_larg=calc_doubling_rate(pd_result_larg)
pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)


the test slope is: [2.]
If mode is 'interp', window_length must be less than or equal to the size of x.


In [5]:
pd_result_larg.to_csv('/Users/pawnesh/ws/covid_data_science/covid_19_data_stats/data/raw/COVID-19/processed/COVID_final_set.csv',sep=';',index=False)
print(pd_result_larg[pd_result_larg['country']=='Germany'].tail())

       index_x       date  country   confirmed  people_vaccinated  \
69473    69473 2022-07-19  Germany  29994679.0         64721546.0   
69474    69474 2022-07-20  Germany  30131303.0         64722459.0   
69475    69475 2022-07-21  Germany  30239122.0         64723588.0   
69476    69476 2022-07-22  Germany  30331131.0         64725416.0   
69477    69477 2022-07-23  Germany  30331133.0                NaN   

       confirmed_filtered  index_y  confirmed_DR  confirmed_filtered_DR  
69473          29982354.6    69473    197.866127             303.523866  
69474          30109983.0    69474    216.071584             253.222339  
69475          30205473.6    69475    246.451740             269.804637  
69476          30292747.2    69476    302.598755             330.510402  
69477          30380020.8    69477    658.626947             347.100924  


### 4 Visual Board


In [ ]:
# %load src/visualization/visualize.py
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())
df_input_large=pd.read_csv('/Users/pawnesh/ws/covid_data_science/covid_19_data_stats/data/raw/COVID-19/processed/COVID_final_set.csv',sep=';')
df_sir=pd.read_csv("/Users/pawnesh/ws/covid_data_science/covid_19_data_stats/data/processed/sir_data.csv", sep=";")

fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data
     Full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['United States', 'Germany','Italy'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
        {'label': 'SIR Model', 'value': 'sir_value'},
    ],
    value='confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    elif 'sir_value' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Population infection'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['country']==each]
        
        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        elif show_doubling=='sir_value':
            df_sir_filter = df_sir[[f"{each}", f"{each}__infections", f"{each}__fitted"]]
            traces.append(dict(x=df_sir_filter[f"{each}"],
                                y=df_sir_filter[f"{each}__infections"],
                                mode='markers',
                                opacity=0.9,
                                name=each
                        )
                )
            traces.append(dict(x=df_sir_filter[f"{each}"],
                                y=df_sir_filter[f"{each}__fitted"],
                                mode='lines',
                                opacity=0.9,
                                name=each
                        )
                )
            
        else:
            df_plot=df_plot[['country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
            #print(show_doubling)


            traces.append(dict(x=df_plot.date,
                                    y=df_plot[show_doubling],
                                    mode='markers+lines',
                                    opacity=0.9,
                                    name=each
                            )
                    )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)

/Users/pawnesh/ws/covid_data_science/covid_19_data_stats/notebooks
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
